In [ ]:
from typing import Tuple
K.<a> = QQ[2*cos(2*pi/7)]

def rotate(d: Tuple[K, K], n: Zmod(7) = 1) -> Tuple[K, K]:
    x, y = d
    if n == 0:
        return (x, y)
    if n == 1:
        return ((a)*x + y + 1, -x)
    if n == 2:
        return ((a^2 - 1)*x + (a)*y + (a + 1), (-a)*x - y - 1)
    if n == 3:
        return ((-a^2 + 1)*x + (a^2 - 1)*y + (a^2 + a), (-a^2 + 1)*x + (-a)*y + (-a - 1))
    if n == 4:
        return ((-a)*x + (-a^2 + 1)*y + (a + 1), (a^2 - 1)*x + (-a^2 + 1)*y + (-a^2 - a))
    if n == 5:
        return (-x + (-a)*y + 1, (a)*x + (a^2 - 1)*y + (-a - 1))
    return (-y, x + (a)*y - 1)


DOTS = [rotate((0, 0), i) for i in range(7)]


def ob(d: Tuple[K, K]) -> Tuple[K, K]:
    x, y = d
    for i in Zmod(7):
        if x > 0 and y > 0:
            return rotate((-x, -y), -i)
        x, y = rotate((x, y))
    raise NotImplementedError()


def ob_inv(d: Tuple[K, K]) -> Tuple[K, K]:
    x, y = d
    for i in Zmod(7):
        if x < 0 and y < 0:
            return rotate((-x, -y), -i)
        x, y = rotate((x, y))
    return (x, y)


# находит номер области, соответствующий номеру точки, относительно
# которой отражает ob
def locate_dot(dot: Tuple[K, K]):
    x, y = dot
    for i in Zmod(7):
        if x > 0 and y > 0:
            return f"{i}"
        x, y = rotate((x, y))

    raise NotImplementedError()


# генератор орбиты
# лучше глянуть пример в examples.ipynb
def orbit_ob(d: Tuple[K, K], n):
    for _ in range(n):
        yield d
        d = ob(d)


# маршрут
def trace_ob(d: Tuple[K, K], n):
    return "".join([locate_dot(i) for i in orbit_ob(d, n)])


def r2k(r: RR) -> K:
    return K(r.nearby_rational(max_error=0.0001))

SIN = sin(2*pi/7)
COS = cos(2*pi/7)
CTG = cot(2*pi/7)

def n2o(d: Tuple[K, K]) -> Tuple[RR, RR]:
    x, y = map(RR, d)
    return (x, RR(COS*x + SIN*y))


def o2n(d: Tuple[RR, RR]) -> Tuple[K, K]:
    x, y = d
    return (r2k(x), r2k(RR(-CTG*x + y/SIN)))

In [ ]:
import plotly.graph_objects as go
from sage.geometry.polyhedron.plot import cyclic_sort_vertices_2d


def intersect_polygons(poly1, poly2):
    poly3 = poly1.intersection(poly2)
    
    if len(poly3.vertices()) > 2:
        return {poly3}
    
    return set()
    

def router(d: Tuple[K, K]):
    try:
        return trace_ob(d, 28)
    except NotImplementedError:
        return "0"

def plot_polygon(_poly):
    poly = tuple(map(tuple, cyclic_sort_vertices_2d(_poly.vertices())))
    N = K(len(poly))
    x, y = zip(*poly)
    it = router((sum(x)/N, sum(y)/N))
    poly = tuple(poly) + (poly[0],)
    x_, y_ = zip(*map(n2o, poly))
    return go.Scatter(x=x_, y=y_, name=it, fill="toself", mode="lines")

def rotated_polygons(polygons):
    for polygon in polygons:
        for i in range(1, 7):
            yield Polyhedron(vertices=tuple(rotate(tuple(d), i) for d in polygon.vertices()))

In [ ]:
from tqdm.notebook import tqdm_notebook

SIZE = 10
ITERATIONS = 5

polygons = {Polyhedron(vertices=((0, 0), (0, SIZE), (SIZE, 0)))}

for i in tqdm_notebook(range(ITERATIONS)):
    new_polys = set()
    for rpoly in rotated_polygons(polygons):
        for poly in polygons:
            new_polys.update(intersect_polygons(poly, -rpoly))
    polygons = new_polys

In [ ]:
new_polys = set()
for rpoly in rotated_polygons(polygons):
    for poly in polygons:
        new_polys.update(intersect_polygons(poly, -rpoly))
polygons = new_polys

In [ ]:
layout = go.Layout(width=800, height=800)

f = go.FigureWidget([plot_polygon(Polyhedron(vertices=DOTS))], layout=layout)

for polygon in polygons:
    f.add_trace(plot_polygon(polygon))

for polygon in rotated_polygons(polygons):
    f.add_trace(plot_polygon(polygon))

f

In [ ]:
f.write_html('7gon_final_form.html')